# IC415 - TP N°7 - Ejercicio 3

## Problema de optimización de Hospitales 

### El objetivo del algoritmo es determinar la ubicación (de la manera más óptima posible) de 9 hospitales en territorio misionero. 

Librerias a utilizar:

In [1]:
import pandas as pd
from random import uniform
from geopy.distance import great_circle
import folium
from math import exp 

ModuleNotFoundError: No module named 'pandas'

In [2]:
class Solucion():

    def __init__(self, radio):
        self.hospitales = []
        self.radio = radio #impone la restriccion para la distancia minima entre hospitales

    def agregarHospital(self, hospital):
        self.hospitales.append(hospital)

    def ultimoHospital(self): #entrega el ultimo hospital cargado
        if self.hospitales:
            return self.hospitales[-1]
        else:
            return None

    def leerHospitales(self):
        return self.hospitales

    def cambiarHospital(self, hospital):
        self.hospitales.pop()
        self.hospitales.append(hospital)

    def validez(self, posibleHospital) -> bool: #se utilizara para la restriccion de radios de hospitales
        valido=True

        if len(self.hospitales)>1:
        
            for i in range(0, len(self.hospitales) - 1):
                
                distancia = float(great_circle(posibleHospital.coordenadas, self.hospitales[i].coordenadas).kilometers)

                if distancia < self.radio:
                    #print(f"Esta es la distancia que se obtuvo que es menor al radio: {distancia}")
                    valido = False
        
        return valido 

class Ubicacion():
    def __init__(self, lat, lon):
        self.coordenadas = (lat, lon) #tupla de coordenadas

class Municipio(Ubicacion):
    def __init__(self, nomMun, lat, lon, den):
        super().__init__(lat, lon)
        self.municipio = nomMun
        self.densidad = den

    def __str__(self):
        return f'Municipio: {self.municipio} | Latitud: {self.coordenadas[0]} | Longitud: {self.coordenadas[1]} | Densidad: {self.densidad}'

class Hospital(Ubicacion):
    def __init__(self, lat, lon, municipios):
        super().__init__(lat, lon)
        self.dentroMnes = self.dentroTerritorio() #de tipo Bool.
        self.municipios = municipios #lista de municipios con informacion a utilizar para calcular la relacion
        self.relDistDen = self.calRelDistDen() #de tipo float
        
    def __str__(self):
        return f'Latitud: {self.coordenadas[0]} | Longitud: {self.coordenadas[1]}'

    def dentroTerritorio(self) -> bool: # a traves de funciones por tramos, analiza si el hospital esta dentro de Misiones
        #1er tramo
        if self.coordenadas[1] >= -56.062706 and self.coordenadas[1] < -55.748588:
            if self.coordenadas[0] >= ((-1.910673 * self.coordenadas[1]) - 134.60372) and self.coordenadas[0] <= ((-0.372665 * self.coordenadas[1]) - 48.220191):
                return True
        #2do tramo
        elif self.coordenadas[1] >= -55.748588 and self.coordenadas[1] < -55.563428:
            if self.coordenadas[0] >= (-28.154587) and self.coordenadas[0] <= ((1.641186 * self.coordenadas[1]) + 64.049196):
                return True
        #3er tramo
        elif self.coordenadas[1] >= -55.563428 and self.coordenadas[1] < -55.125114:
            if self.coordenadas[0] >= ((0.58064 * self.coordenadas[1]) + 4.107762) and self.coordenadas[0] <= ((0.491182 * self.coordenadas[1]) + 0.151032):
                return True
        #4to tramo
        elif self.coordenadas[1] >= -55.125114 and self.coordenadas[1] < -54.606529:
            if self.coordenadas[0] >= ((0.58064 * self.coordenadas[1]) + 4.107762) and self.coordenadas[0] <= ((1.358984 * self.coordenadas[1]) + 47.988751):
                return True
        #5to tramo
        elif self.coordenadas[1] >= -54.606529 and self.coordenadas[1] < -53.820817:
            if self.coordenadas[0] >= ((0.58064 * self.coordenadas[1]) + 4.107762) and self.coordenadas[0] <= (-25.501328):
                return True
        #6to tramo
        elif self.coordenadas[1] >= -53.820817 and self.coordenadas[1] <= -53.639870:
            if self.coordenadas[0] >= ((-1.910673 * self.coordenadas[1]) - 134.60372) and self.coordenadas[0] <= ((-1.326287 * self.coordenadas[1]) - 97.362521):
                return True
        #esta fuera de Misiones
        return False

    def calRelDistDen(self):
        #funcion matematica sumatoria que relaciona distancia y densidad
        k = 50
        num = den = 0.0

        for m in municipios:
            dist = float(great_circle(self.coordenadas, m.coordenadas).kilometers)

            num += dist
            den += m.densidad * exp(-(dist/k))
        
        return num / den


In [3]:
misiones = pd.read_csv('Datos_Misiones.csv', sep=";")
municipios = []

misiones

count = 0

for mun in misiones["Municipios"]:
    nomMun = misiones["Municipios"][count]
    lat = misiones["Latitud"][count]
    lon = misiones["Longitud"][count]
    den = misiones["Densidad (hab/km^2)"][count]

    m = Municipio(nomMun, lat, lon, den)
    municipios.append(m)
    count += 1

#for m in municipios:
#    print(m)

Generacion de coordenadas de los 9 hospitales:

In [4]:
solucion = Solucion(50.0) # Radio (float)  

for i in range(9):
    prueba = 1
    numPruebas = 700

    while prueba <= numPruebas:
    
        lat = uniform(-28.154587 , -25.501323)
        lon = uniform(-56.062706 , -53.639878)

        h = Hospital(lat, lon, municipios) 
    
        if h.dentroMnes:
            
            if solucion.validez(h):
                
                if prueba == 1:
                    solucion.agregarHospital(h)
                
                else: 
                    ultimoHospital = solucion.ultimoHospital()
                    #se comparan los valores de los valores de relacion distancia/densidad, si es menor, lo guarda como solucion
                    if h.relDistDen <= ultimoHospital.relDistDen:
                        solucion.cambiarHospital(h)
                
                prueba += 1


Coordenadas de los 9 hospitales:

In [5]:
hospitales = solucion.leerHospitales()

print("Coordenadas de los 9 hospitales:")

for h in hospitales:
    print(h)

Coordenadas de los 9 hospitales:
Latitud: -27.497784750634423 | Longitud: -55.213568140815305
Latitud: -27.404614022737796 | Longitud: -55.86052233238298
Latitud: -27.07190228436806 | Longitud: -55.03139449602411
Latitud: -26.656842205783555 | Longitud: -54.787821395676254
Latitud: -27.81152158938981 | Longitud: -55.64391514765021
Latitud: -27.41781005342528 | Longitud: -54.696231733027126
Latitud: -27.00155418344556 | Longitud: -54.39419514049877
Latitud: -26.299951785986657 | Longitud: -54.48055888897567
Latitud: -26.620938012695717 | Longitud: -54.06513470568306


In [6]:
map = folium.Map(location=[-26.7952403,-54.4926572], tiles = "Stamen Toner", zoom_start = 8)

#se añaden los municipios al mapa 
for mun in municipios:
    folium.Marker([mun.coordenadas[0], mun.coordenadas[1]], popup = f"{mun.municipio}", icon=folium.Icon(color="green")).add_to(map)

#se añaden los hospitales solucion al mapa 
for i in range(9):
    h = hospitales[i]
    folium.Marker([h.coordenadas[0], h.coordenadas[1]], popup = f"Hospital_{i+1}", icon=folium.Icon(color="red" , icon = "")).add_to(map)

map